# Prepare Environment

**Install pyspark package**

The first step before running spark is to install the ``pyspark`` package, because the package is not included in the default Google Colab setup.
This step needs to be done every time we open a new session

In [ ]:
!pip install pyspark

     |████████████████████████████████| 281.4 MB 34 kB/s 
     |████████████████████████████████| 198 kB 46.2 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=1e4f01629fdb1926823d7462e6508631fc0771de8672683330f0e89c5673e772
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


**b. Create spark session**

In [ ]:
import pyspark
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder.appName('DataFrame Basics').getOrCreate()



We will perform some simple data engineering with sales data. First we will load the clean data, and then we will use 'dirty' data and do some cleansing and enrichment.

#Latihan data

Data covid, ada 3 file

- data harian covid :
seq, dateRep, cases, deaths, geoId, cnum_14days_per_100K

- data referensi1 :
countriesAndTerritories, geoId, countryterritoryCode, popData2019, continentExp

- data referensi 2 :
Country,	Alpha-2 code,	Alpha-3 code,	Numeric code,	Latitude (average),	Longitude (average)


##1. Data Harian Covid

Data utama
Ada data duplikat, ada data incomplete, ada data yang tidak standar

In [ ]:
#Data harian covid

!wget https://www.dropbox.com/s/9le71jjcpdqzzk9/datacovid_dup1.csv

--2022-03-29 23:19:25--  https://www.dropbox.com/s/9le71jjcpdqzzk9/datacovid_dup1.csv
Resolving www.dropbox.com (www.dropbox.com)... 162.125.9.18, 2620:100:6019:18::a27d:412
Connecting to www.dropbox.com (www.dropbox.com)|162.125.9.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/9le71jjcpdqzzk9/datacovid_dup1.csv [following]
--2022-03-29 23:19:25--  https://www.dropbox.com/s/raw/9le71jjcpdqzzk9/datacovid_dup1.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc062ad5c9910dd984a8bdc29169.dl.dropboxusercontent.com/cd/0/inline/BibOBcumQGVd2q9V0t1YAbZd1_UJaeOepHcyVw-0MDOEtjhK8EbH1V1ATKYTIzcrlnTZlwYCwIumPOLp0cFT00nZaShR8iatWrHLPpaZey8ExdcYslK_jfd8aaJaHS0BycdTfDde867_0gmznXCmkuWOhmwMrpvm_K30TacFl0HPNw/file# [following]
--2022-03-29 23:19:26--  https://uc062ad5c9910dd984a8bdc29169.dl.dropboxusercontent.com/cd/0/inline/BibOBcumQGVd2q9V0t1YAbZd1_UJaeOepHcyVw-0MDOEtjhK8E

In [ ]:
df = spark.read.csv("datacovid_dup1.csv",header=True,inferSchema=True)

In [ ]:
df.show()
df.printSchema()

+---+----------+-----+------+-----+--------------------+
|_c0|   dateRep|cases|deaths|geoId|cnum_14days_per_100K|
+---+----------+-----+------+-----+--------------------+
|  0|08/11/2020|126.0|   6.0|   AF|          3.65650829|
|  1|07/11/2020| 58.0|   2.0|   AF|          3.53821723|
|  2|06/11/2020| 40.0|   0.0|   AF|           3.5461033|
|  3|05/11/2020|121.0|   6.0|   AF|          3.74588377|
|  4|04/11/2020| 86.0|   4.0|   AF|          3.78268543|
|  5|03/11/2020| 95.0|   3.0|   AF|          3.78794281|
|  6|02/11/2020|132.0|   5.0|   AF|          3.76691329|
|  7|01/11/2020| 76.0|   0.0|   AF|          3.57501889|
|  8|31/10/2020|157.0|   4.0|   AF|          3.55398937|
|  9|30/10/2020|123.0|   3.0|   AF|          3.26483343|
| 10|29/10/2020|  0.0|   0.0|   AF|          2.94150452|
| 11|28/10/2020|113.0|   7.0|   AF|          3.02562261|
| 12|27/10/2020|199.0|   8.0|   AF|          2.90207416|
| 13|26/10/2020| 65.0|   3.0|   AF|          2.71806583|
| 14|25/10/2020| 81.0|   4.0|  

##Data Referensi 1

Tidak ada data duplikat, semua data bisa dikorelasikan dengan data utama (data harian covid)

In [ ]:
!wget https://www.dropbox.com/s/attxr4nalhpakti/countries3.csv

--2022-03-29 23:19:36--  https://www.dropbox.com/s/attxr4nalhpakti/countries3.csv
Resolving www.dropbox.com (www.dropbox.com)... 162.125.4.18, 2620:100:601d:18::a27d:512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.4.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/attxr4nalhpakti/countries3.csv [following]
--2022-03-29 23:19:36--  https://www.dropbox.com/s/raw/attxr4nalhpakti/countries3.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc4cf799f8e35dabf486a71b80fe.dl.dropboxusercontent.com/cd/0/inline/Bia0bXk7WdD8oMcbr1AiLUQegtGDsqw307tKAAm59MdI0N-B_Ij5MxzwUFxPCbZPJ1FhlRR8FvCFlTOJkXd6R--WK45xR-pXX0KMcRjB41Zq2z59XzTdAcRkZw3LH6LK0Dke08nK3-XDw9n37Kt85QNu8NtoeptQv35o73Bu7RkmGw/file# [following]
--2022-03-29 23:19:36--  https://uc4cf799f8e35dabf486a71b80fe.dl.dropboxusercontent.com/cd/0/inline/Bia0bXk7WdD8oMcbr1AiLUQegtGDsqw307tKAAm59MdI0N-B_Ij5MxzwUFxPCb

In [ ]:
dfr1 = spark.read.csv("countries3.csv",header=True,inferSchema=True)

In [ ]:
dfr1.show()
dfr1.printSchema()

+---+-----------------------+-----+--------------------+------------+------------+
|_c0|countriesAndTerritories|geoId|countryterritoryCode| popData2019|continentExp|
+---+-----------------------+-----+--------------------+------------+------------+
|  0|           Cote_dIvoire|   CI|                 CIV| 2.5716554E7|      Africa|
|  1|                Lebanon|   LB|                 LBN|   6855709.0|        Asia|
|  2|                Armenia|   AM|                 ARM|   2957728.0|      Europe|
|  3|                Bolivia|   BO|                 BOL| 1.1513102E7|     America|
|  4|          Guinea_Bissau|   GW|                 GNB|   1920917.0|      Africa|
|  5|                Hungary|   HU|                 HUN|   9772756.0|      Europe|
|  6|                 Taiwan|   TW|             CNG1925| 2.3773881E7|        Asia|
|  7|   United_States_of_...|   US|                 USA|3.29064917E8|     America|
|  8|             Tajikistan|   TJ|                 TJK|   9321023.0|        Asia|
|  9

## Data referensi 2

Ada data duplikat
Ada data yang tidak sama dengan data utama dan referensi 1

In [ ]:
!wget https://www.dropbox.com/s/qdv4ipzdppam6hz/countries_codes_and_coordinates.csv

--2022-03-29 23:19:37--  https://www.dropbox.com/s/qdv4ipzdppam6hz/countries_codes_and_coordinates.csv
Resolving www.dropbox.com (www.dropbox.com)... 162.125.4.18, 2620:100:601c:18::a27d:612
Connecting to www.dropbox.com (www.dropbox.com)|162.125.4.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/qdv4ipzdppam6hz/countries_codes_and_coordinates.csv [following]
--2022-03-29 23:19:37--  https://www.dropbox.com/s/raw/qdv4ipzdppam6hz/countries_codes_and_coordinates.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc7c9fbf98236feb293a8d2a28ba.dl.dropboxusercontent.com/cd/0/inline/BibgmLznl8nU1Jdul6FY4E0H7jbTU_jsvSgvvFYbYJNuGFr649wfUn2A9yI4aCzrIdx2kth2CZU3n5WGYSLIjbwVTbDL77Nmbbc9KIyH0tB67aDAeBSSNdcCGUlts_7_UqsdNfx8wy30N_6BMeDV_T3gHgLSN8OsF6tQxtUGFysPew/file# [following]
--2022-03-29 23:19:37--  https://uc7c9fbf98236feb293a8d2a28ba.dl.dropboxusercontent.com/cd/0/inline

In [ ]:
dfr2 = spark.read.csv("countries_codes_and_coordinates.csv",header=True,inferSchema=True,quote='"',ignoreLeadingWhiteSpace=True)

In [ ]:
dfr2.show()
dfr2.printSchema()

+-------------------+------------+------------+------------+------------------+-------------------+
|            Country|Alpha-2 code|Alpha-3 code|Numeric code|Latitude (average)|Longitude (average)|
+-------------------+------------+------------+------------+------------------+-------------------+
|        Afghanistan|          AF|         AFG|           4|              33.0|               65.0|
|            Albania|          AL|         ALB|           8|              41.0|               20.0|
|            Algeria|          DZ|         DZA|          12|              28.0|                3.0|
|     American Samoa|          AS|         ASM|          16|          -14.3333|             -170.0|
|            Andorra|          AD|         AND|          20|              42.5|                1.6|
|             Angola|          AO|         AGO|          24|             -12.5|               18.5|
|           Anguilla|          AI|         AIA|         660|             18.25|           -63.1667|


### Join dengan data utama

In [ ]:
left_join = df.join(dfr2, df.geoId == dfr2['Alpha-2 code'],how='left')

In [ ]:
left_join.show()

+---+----------+-----+------+-----+--------------------+-----------+------------+------------+------------+------------------+-------------------+
|_c0|   dateRep|cases|deaths|geoId|cnum_14days_per_100K|    Country|Alpha-2 code|Alpha-3 code|Numeric code|Latitude (average)|Longitude (average)|
+---+----------+-----+------+-----+--------------------+-----------+------------+------------+------------+------------------+-------------------+
|  0|08/11/2020|126.0|   6.0|   AF|          3.65650829|Afghanistan|          AF|         AFG|           4|              33.0|               65.0|
|  1|07/11/2020| 58.0|   2.0|   AF|          3.53821723|Afghanistan|          AF|         AFG|           4|              33.0|               65.0|
|  2|06/11/2020| 40.0|   0.0|   AF|           3.5461033|Afghanistan|          AF|         AFG|           4|              33.0|               65.0|
|  3|05/11/2020|121.0|   6.0|   AF|          3.74588377|Afghanistan|          AF|         AFG|           4|           